In [1]:
import numpy as np
import xarray as xr
import pandas as pd

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import matplotlib

In [2]:
lon_start = [104.0, 104.0, 104.0, 104.0, 104.0, 104.0, 104.0, 104.0, 104.0, 104.0]
lon_end = [115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0]
lat_start = [5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5]
lat_end = [7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]

# lon_start = [110.0, 108.0, 101.0, 97.0, 94.0, 91.0, 85.0]
# lon_end = [126.0, 124.0, 117.0, 113.0, 110.0, 107.0, 103.0]
# lat_start = [5.0, 5.0, 5.0, 5.0, 5.0, 5.5, 5.5]
# lat_end = [6.5, 6.5, 6.5, 6.5, 6.5, 7.0, 7.0]

In [3]:
it = 0

vvel_6km = []
heating_rate_6km = []
r_6km = []

uwind_upper = []
uwind_mid = []
uwind_lower = []

for Tp in [12,24,36,48,60,72,84,96,108,120]:
    
    time = int(Tp)-12
    gl_pe='/nobackup/earshar/borneo/case_20181021T1200Z_N768/nc/umglaa_pe{0:03d}.nc'.format(time)
    gl_pc='/nobackup/earshar/borneo/case_20181021T1200Z_N768/nc/umglaa_pc{0:03d}.nc'.format(time)
    gl_pb='/nobackup/earshar/borneo/case_20181021T1200Z_N768/nc/umglaa_pb{0:03d}.nc'.format(time)
    
    input_data_pe = xr.open_dataset(gl_pe) # vertical velocity 
    input_data_pc = xr.open_dataset(gl_pc, drop_variables=['unspecified_5','unspecified_6',   # heating rate 
                                                           'unspecified_9','unspecified_10'])
    input_data_pb = xr.open_dataset(gl_pb)
        
    var = input_data_pc['unspecified'].squeeze('t').sel(hybrid_ht=6000, 
                                                        method='nearest').sel(latitude=slice(lat_start[it], 
                                                        lat_end[it])).sel(longitude=slice(lon_start[it], 
                                                        lon_end[it]))
    
    if Tp == 12:
        vvel = input_data_pe.dz_dt[1,:,:,:].sel(hybrid_ht_1=6000, 
                                                method='nearest').sel(latitude=slice(lat_start[0], 
                                                lat_end[0], )).sel(longitude_1=slice(lon_start[0], lon_end[0]))
        
        r = input_data_pb.q[1,:,:,:].sel(hybrid_ht=6000, 
                                        method='nearest').sel(latitude=slice(lat_start[0], 
                                        lat_end[0], )).sel(longitude=slice(lon_start[0], lon_end[0]))

        uwind_u = input_data_pe.u[1,:,:,:].sel(hybrid_ht=6000, 
                                               method='nearest').sel(latitude=slice(lat_start[0], 
                                               lat_end[0])).sel(longitude=slice(lon_start[0], lon_end[0]))
        
        uwind_m = input_data_pe.u[1,:,:,:].sel(hybrid_ht=slice(4500,5000)).sel(latitude=slice(lat_start[0], 
                                               lat_end[0])).sel(longitude=slice(lon_start[0], lon_end[0]))

        uwind_l = input_data_pe.u[1,:,:,:].sel(hybrid_ht=1400, 
                                               method='nearest').sel(latitude=slice(lat_start[0], 
                                               lat_end[0])).sel(longitude=slice(lon_start[0], lon_end[0]))
    else:
        vvel = input_data_pe.dz_dt.squeeze('t').sel(hybrid_ht=6000, 
                                                    method='nearest').sel(latitude=slice(lat_start[it], 
                                                    lat_end[it])).sel(longitude=slice(lon_start[it], lon_end[it]))
        
        r = input_data_pb.q.squeeze('t').sel(hybrid_ht=6000, 
                                            method='nearest').sel(latitude=slice(lat_start[it], 
                                            lat_end[it])).sel(longitude=slice(lon_start[it], lon_end[it]))
        
        uwind_u = input_data_pe.u.squeeze('t').sel(hybrid_ht_1=6000, 
                                                   method='nearest').sel(latitude=slice(lat_start[it], 
                                                   lat_end[it])).sel(longitude_1=slice(lon_start[it], lon_end[it]))

        uwind_m = input_data_pe.u.squeeze('t').sel(hybrid_ht_1=slice(4500,5000)).sel(latitude=slice(lat_start[it], 
                                                   lat_end[it])).sel(longitude_1=slice(lon_start[it], lon_end[it]))

        uwind_l = input_data_pe.u.squeeze('t').sel(hybrid_ht_1=1400, 
                                                   method='nearest').sel(latitude=slice(lat_start[it], 
                                                   lat_end[it])).sel(longitude_1=slice(lon_start[it], lon_end[it]))
        
    heating_rate = 0
    vars_list = ['unspecified', 'unspecified_1', 'unspecified_2', 'unspecified_3',
                 'unspecified_4', 'unspecified_7','unspecified_8']
    for var in vars_list:
        heating_rate = heating_rate + input_data_pc[var].squeeze('t').sel(hybrid_ht=6000, 
                                                        method='nearest').sel(latitude=slice(lat_start[it], 
                                                        lat_end[it])).sel(longitude=slice(lon_start[it], 
                                                        lon_end[it]))
    heating_rate = heating_rate / 3600. # uncomment to convert to K/s (from K/h)
    
    vvel_1d = vvel.data.reshape(vvel.size)
    heating_rate_1d = heating_rate.data.reshape(heating_rate.size)
    r_1d = r.data.reshape(r.size)
    
    vvel_6km.append(vvel_1d)
    heating_rate_6km.append(heating_rate_1d)
    r_6km.append(r_1d)
    
    uwind_upper_1d = uwind_u.data.reshape(uwind_u.size)
    uwind_upper.append(uwind_upper_1d)
    
    uwind_mid_1d = uwind_m.data.reshape(uwind_m.size)
    uwind_mid.append(uwind_mid_1d)   
    
    uwind_lower_1d = uwind_l.data.reshape(uwind_l.size)
    uwind_lower.append(uwind_lower_1d)
    
    it += 1

In [4]:
vvel_all = [val for vvel_6km in vvel_6km for val in vvel_6km]
heating_rate_all = [val for heating_rate_6km in heating_rate_6km for val in heating_rate_6km]
r_all = [val for r_6km in r_6km for val in r_6km]

uwind_upper_all = [val for uwind_upper in uwind_upper for val in uwind_upper]
uwind_mid_all = [val for uwind_mid in uwind_mid for val in uwind_mid]
uwind_lower_all = [val for uwind_lower in uwind_lower for val in uwind_lower]

In [5]:
# calculate mean zonal wind over specified region
uwind_upper_mean = sum(uwind_upper_all) / len(uwind_upper_all)
uwind_mid_mean = sum(uwind_mid_all) / len(uwind_mid_all)
uwind_lower_mean = sum(uwind_lower_all) / len(uwind_lower_all)

In [7]:
# uwind_upper_mean, uwind_mid_mean, uwind_lower_mean

In [13]:
# calculate rescaled vertical velocity 
th_ref = 300.0
g = 9.81
latitude = 6.25 * (np.pi / 180.)
f0 = 1.4584E-4 * np.sin(latitude)

vvel_scale = (np.array(vvel_all) * np.array(r_all) * th_ref * f0) / g

In [14]:
vvel_scale = vvel_scale * 100000.

In [15]:
vvel_plot = vvel_scale

In [16]:
# produce scatter plot using data from all times 
fig, ax = plt.subplots(figsize=(12, 9))
ax.scatter(vvel_plot, heating_rate_all, marker=".")
# ax.set_xlabel(r'Rescaled vertical velocity $\mathregular{(m\,s^{-1})}$',fontsize='large')
ax.set_xlabel(r'Rescaled vertical velocity $\mathregular{(K\,s^{-1})}$',fontsize='large')
ax.set_ylabel(r'Heating rate $\mathregular{(K\,s^{-1})}$',fontsize='large')
gradient, intercept = np.polyfit(vvel_plot, heating_rate_all, 1)
plt.plot(vvel_plot, (np.array(vvel_plot)*gradient + intercept), color='black', linestyle='--', linewidth=2)
plt.grid(color='grey', linestyle='--', linewidth=1)
plt.savefig(f'scatter_plot.png')
plt.show()
plt.close()

In [17]:
gradient

0.0036795548988027443